In [72]:
# Import all the necessary modules
import os
import sys
import os, sys
# from .../research/notebooks -> go up two levels to repo root
repo_root = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as mtick
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score 
import pandas_datareader as pdr
import math
import datetime as dt
from datetime import datetime, timezone
import itertools
import ast
import yfinance as yf
import seaborn as sn
import yaml
import requests
from pathlib import Path
from IPython.display import display, HTML
from strategy_signal.trend_following_signal import (
    apply_jupyter_fullscreen_css, get_trend_donchian_signal_for_portfolio_with_rolling_r_sqr_vol_of_vol
)
from portfolio.strategy_performance import (calculate_sharpe_ratio, calculate_calmar_ratio, calculate_CAGR, calculate_risk_and_performance_metrics,
                                          calculate_compounded_cumulative_returns, estimate_fee_per_trade, rolling_sharpe_ratio)
from utils import coinbase_utils as cn
from portfolio import strategy_performance as perf
from sizing import position_sizing_binary_utils as size_bin
from sizing import position_sizing_continuous_utils as size_cont
from strategy_signal import trend_following_signal as tf
%matplotlib inline

In [20]:
import importlib
importlib.reload(cn)
importlib.reload(perf)
importlib.reload(tf)
importlib.reload(size_bin)
importlib.reload(size_cont)

<module 'sizing.position_sizing_continuous_utils' from '/Users/adheerchauhan/Documents/git/trend_following/sizing/position_sizing_continuous_utils.py'>

In [22]:
import warnings
warnings.filterwarnings('ignore')
pd.set_option('Display.max_rows', None)
pd.set_option('Display.max_columns',None)
apply_jupyter_fullscreen_css()

## Coinbase Utility Functions

In [74]:
def get_coinbase_historical_price_data(
    client,
    ticker,
    start_timestamp,
    end_timestamp,
    granularity="ONE_DAY",
    retries=3,
    delay=5,
):
    """
    Generic candle puller for Coinbase Advanced Trade RESTClient.get_candles().

    granularity examples:
      ONE_MINUTE, FIVE_MINUTE, FIFTEEN_MINUTE, THIRTY_MINUTE,
      ONE_HOUR, TWO_HOUR, FOUR_HOUR, SIX_HOUR, ONE_DAY
    """
    attempts = 0
    while attempts < retries:
        try:
            candle_list = client.get_candles(
                product_id=ticker,
                start=int(start_timestamp),
                end=int(end_timestamp),
                granularity=granularity,
            ).candles

            if not candle_list:
                cols = ["open", "high", "low", "close", "volume"]
                return pd.DataFrame(columns=cols).rename_axis("date")

            candle_data = []
            for c in candle_list:
                candle_data.append(
                    {
                        "date": c["start"],  # epoch seconds
                        "low": float(c["low"]),
                        "high": float(c["high"]),
                        "open": float(c["open"]),
                        "close": float(c["close"]),
                        "volume": float(c["volume"]),
                    }
                )

            df = pd.DataFrame(candle_data)
            if df.empty or "date" not in df.columns:
                cols = ["open", "high", "low", "close", "volume"]
                return pd.DataFrame(columns=cols).rename_axis("date")

            # epoch seconds -> tz-aware UTC -> drop tz (tz-naive UTC)
            s = pd.to_datetime(pd.to_numeric(df["date"], errors="coerce"), unit="s", utc=True).dt.tz_localize(None)

            # Only normalize for daily bars; keep intraday timestamps intact
            if granularity == "ONE_DAY":
                s = s.dt.normalize()

            df["date"] = s
            df = df.set_index("date").sort_index().rename_axis("date")

            return df

        except requests.exceptions.ConnectionError as e:
            print(f"Connection error: {e}. Retrying in {delay} seconds...")
            attempts += 1
            time.sleep(delay)

    raise Exception("Max retries exceeded. Could not connect to Coinbase API.")


In [76]:
def save_historical_crypto_prices_from_coinbase(
    ticker,
    user_start_date=False,
    start_date=None,
    end_date=None,
    save_to_file=False,
    portfolio_name="Default",
    granularity="ONE_DAY",
):
    """
    Pull historical candles for a single ticker at the requested granularity.

    Note: Coinbase candle endpoints have request caps (commonly 300 candles per call),
    so we chunk requests.
    """
    client = cn.get_coinbase_rest_api_client(portfolio_name=portfolio_name)

    if user_start_date:
        start_date = pd.Timestamp(start_date)
    else:
        start_date = cn.coinbase_start_date_by_ticker_dict.get(ticker)
        start_date = pd.Timestamp(start_date)
        if start_date is None:
            print(f"Start date for {ticker} is not included in the dictionary!")
            return None

    end_date = pd.Timestamp(end_date)

    # seconds per bar (used to step chunks without gaps)
    granularity_to_seconds = {
        "ONE_MINUTE": 60,
        "FIVE_MINUTE": 300,
        "FIFTEEN_MINUTE": 900,
        "THIRTY_MINUTE": 1800,
        "ONE_HOUR": 3600,
        "TWO_HOUR": 7200,
        "FOUR_HOUR": 14400,
        "SIX_HOUR": 21600,
        "ONE_DAY": 86400,
    }
    bar_sec = granularity_to_seconds.get(granularity)
    if bar_sec is None:
        raise ValueError(f"Unsupported granularity: {granularity}")

    # Keep your old 6-week chunking (works great for ONE_DAY and FOUR_HOUR),
    # but ensure we never step by +1 day when doing intraday.
    temp_start = start_date
    current_end = temp_start

    dfs = []
    while current_end < end_date:
        # 6 weeks is safe for FOUR_HOUR (≈252 candles) under the typical 300 limit :contentReference[oaicite:1]{index=1}
        current_end = pd.to_datetime(temp_start) + dt.timedelta(weeks=6)
        if current_end > end_date:
            current_end = end_date

        start_ts = int(pd.Timestamp(temp_start).timestamp())
        end_ts = int(pd.Timestamp(current_end).timestamp())

        df_chunk = get_coinbase_historical_price_data(
            client=client,
            ticker=ticker,
            start_timestamp=start_ts,
            end_timestamp=end_ts,
            granularity=granularity,
        )
        dfs.append(df_chunk)

        # advance by exactly one bar to avoid duplicates and avoid gaps
        temp_start = pd.to_datetime(current_end) + pd.Timedelta(seconds=bar_sec)

    if not dfs:
        cols = ["open", "high", "low", "close", "volume"]
        return pd.DataFrame(columns=cols).rename_axis("date")

    df = pd.concat(dfs, axis=0)
    df = df[~df.index.duplicated(keep="last")].sort_index()

    # optional: save_to_file logic can stay as you had it (not shown in your snippet)

    return df


In [78]:
cn_ticker_list = cn.coinbase_start_date_by_ticker_dict

In [80]:
cn_ticker_list

{'BTC-USD': '2016-01-01',
 'ETH-USD': '2016-06-01',
 'SOL-USD': '2021-06-01',
 'ADA-USD': '2021-03-01',
 'AVAX-USD': '2021-09-01',
 'DOT-USD': '2021-06-01',
 'ATOM-USD': '2020-01-01',
 'LTC-USD': '2016-09-01',
 'XRP-USD': '2023-06-01',
 'ALGO-USD': '2019-08-01',
 'XLM-USD': '2019-02-01',
 'TON-USD': '2025-11-18',
 'NEAR-USD': '2022-09-01',
 'ICP-USD': '2021-05-10',
 'HBAR-USD': '2022-10-13',
 'SUI-USD': '2023-05-18',
 'CRO-USD': '2021-11-01',
 'APT-USD': '2022-10-19',
 'XTZ-USD': '2019-08-06',
 'EGLD-USD': '2022-12-07',
 'FIL-USD': '2020-12-09',
 'SEI-USD': '2023-08-15',
 'TIA-USD': '2023-11-01',
 'KAVA-USD': '2023-01-19',
 'ROSE-USD': '2022-04-26',
 'MATIC-USD': '2021-02-01',
 'SKL-USD': '2021-02-01',
 'OP-USD': '2022-06-01',
 'ARB-USD': '2023-03-23',
 'POL-USD': '2024-09-04',
 'IMX-USD': '2021-12-09',
 'STRK-USD': '2024-02-21',
 'BLAST-USD': '2024-06-26',
 'ZK-USD': '2024-09-25',
 'LRC-USD': '2020-09-15',
 'ZORA-USD': '2025-04-24',
 'METIS-USD': '2022-06-28',
 'STX-USD': '2022-01-20'

In [164]:
ticker_list = []
exclude_list = ['USDT-USD','DAI-USD','USD1-USD','PAX-USD','MATIC-USD']
for ticker, date in cn_ticker_list.items():
    if (pd.Timestamp(cn_ticker_list[ticker]).date() <= pd.Timestamp('2022-04-01').date()) & (ticker not in exclude_list):
        ticker_list.append(ticker)

In [166]:
print(len(ticker_list))
ticker_list

40


['BTC-USD',
 'ETH-USD',
 'SOL-USD',
 'ADA-USD',
 'AVAX-USD',
 'DOT-USD',
 'ATOM-USD',
 'LTC-USD',
 'ALGO-USD',
 'XLM-USD',
 'ICP-USD',
 'CRO-USD',
 'XTZ-USD',
 'FIL-USD',
 'SKL-USD',
 'IMX-USD',
 'LRC-USD',
 'STX-USD',
 'DOGE-USD',
 'SHIB-USD',
 'LINK-USD',
 'FET-USD',
 'GRT-USD',
 'RNDR-USD',
 'OXT-USD',
 'AIOZ-USD',
 'DIA-USD',
 'KRL-USD',
 'UNI-USD',
 'AAVE-USD',
 'AMP-USD',
 'COMP-USD',
 'MKR-USD',
 'SNX-USD',
 'SUSHI-USD',
 'CRV-USD',
 'BAL-USD',
 '1INCH-USD',
 'MANA-USD',
 'REQ-USD']

In [170]:
%%time
start_date = "2022-04-01"
end_date   = "2024-12-31"

df_list = []
df_by_ticker = {}
ticker_list_len = len(ticker_list)
loop_start = 0
loop_end = 0
counter = 0
while counter < ticker_list_len:
    loop_start = counter
    if counter == 40:
        loop_end = ticker_list_len
    else:
        loop_end = counter + 10
    print(counter, loop_start, loop_end, ticker_list[loop_start: loop_end])
    for t in ticker_list[loop_start: loop_end]:
        df_by_ticker[t] = save_historical_crypto_prices_from_coinbase(
            ticker=t,
            user_start_date=True,
            start_date=start_date,
            end_date=end_date,
            portfolio_name="Default",
            granularity="FOUR_HOUR",
        )
    counter += 10

    # Optional: one combined frame (MultiIndex: ticker, date)
    df_all = pd.concat(df_by_ticker, names=["ticker", "date"]).sort_index()
    df_list.append(df_all)

0 0 10 ['BTC-USD', 'ETH-USD', 'SOL-USD', 'ADA-USD', 'AVAX-USD', 'DOT-USD', 'ATOM-USD', 'LTC-USD', 'ALGO-USD', 'XLM-USD']
10 10 20 ['ICP-USD', 'CRO-USD', 'XTZ-USD', 'FIL-USD', 'SKL-USD', 'IMX-USD', 'LRC-USD', 'STX-USD', 'DOGE-USD', 'SHIB-USD']
20 20 30 ['LINK-USD', 'FET-USD', 'GRT-USD', 'RNDR-USD', 'OXT-USD', 'AIOZ-USD', 'DIA-USD', 'KRL-USD', 'UNI-USD', 'AAVE-USD']
30 30 40 ['AMP-USD', 'COMP-USD', 'MKR-USD', 'SNX-USD', 'SUSHI-USD', 'CRV-USD', 'BAL-USD', '1INCH-USD', 'MANA-USD', 'REQ-USD']
CPU times: user 17.2 s, sys: 1.32 s, total: 18.5 s
Wall time: 2min 1s


In [178]:
len(df_list)

4

In [172]:
df_all.shape

(241229, 5)

In [180]:
df_all.index

MultiIndex([('1INCH-USD', '2022-04-01 00:00:00'),
            ('1INCH-USD', '2022-04-01 04:00:00'),
            ('1INCH-USD', '2022-04-01 08:00:00'),
            ('1INCH-USD', '2022-04-01 12:00:00'),
            ('1INCH-USD', '2022-04-01 16:00:00'),
            ('1INCH-USD', '2022-04-01 20:00:00'),
            ('1INCH-USD', '2022-04-02 00:00:00'),
            ('1INCH-USD', '2022-04-02 04:00:00'),
            ('1INCH-USD', '2022-04-02 08:00:00'),
            ('1INCH-USD', '2022-04-02 12:00:00'),
            ...
            (  'XTZ-USD', '2024-12-29 12:00:00'),
            (  'XTZ-USD', '2024-12-29 16:00:00'),
            (  'XTZ-USD', '2024-12-29 20:00:00'),
            (  'XTZ-USD', '2024-12-30 00:00:00'),
            (  'XTZ-USD', '2024-12-30 04:00:00'),
            (  'XTZ-USD', '2024-12-30 08:00:00'),
            (  'XTZ-USD', '2024-12-30 12:00:00'),
            (  'XTZ-USD', '2024-12-30 16:00:00'),
            (  'XTZ-USD', '2024-12-30 20:00:00'),
            (  'XTZ-USD', '2024-12

In [194]:
ticker_cond = (df_all.index.get_level_values('ticker') == 'ADA-USD')
df_all[ticker_cond].tail(200)

low    high    open   close        volume
ticker  date                                                             
ADA-USD 2024-11-27 20:00:00  1.0012  1.0399  1.0194  1.0123  1.410101e+07
        2024-11-28 00:00:00  0.9889  1.0316  1.0132  1.0071  1.156654e+07
        2024-11-28 04:00:00  0.9909  1.0268  1.0068  0.9968  5.799917e+06
        2024-11-28 08:00:00  0.9818  1.0037  0.9970  0.9976  6.108932e+06
        2024-11-28 12:00:00  0.9705  1.0103  0.9972  0.9775  9.179363e+06
        2024-11-28 16:00:00  0.9774  1.0072  0.9776  1.0021  6.670803e+06
        2024-11-28 20:00:00  0.9955  1.0492  1.0022  1.0323  1.161228e+07
        2024-11-29 00:00:00  1.0278  1.0620  1.0321  1.0489  1.497607e+07
        2024-11-29 04:00:00  1.0357  1.0790  1.0491  1.0458  1.290120e+07
        2024-11-29 08:00:00  1.0435  1.0875  1.0457  1.0651  1.078638e+07
        2024-11-29 12:00:00  1.0542  1.1000  1.0645  1.0761  1.839468e+07
        2024-11-29 16:00:00  1.0390  1.0837  1.0762  1.0521  1.785688e+07
        2024-11-29 20:00:00  1.0450  1.0861  1.0521  1.0761  1.403409e+07
        2024-11-30 00:00:00  1.0505  1.0885  1.0755  1.0697  1.469869e+07
        2024-11-30 04:00:00  1.0604  1.1450  1.0699  1.1299  2.163621e+07
        2024-11-30 08:00:00  1.0778  1.1500  1.1298  1.0956  1.549969e+07
        2024-11-30 12:00:00  1.0716  1.0965  1.0956  1.0929  9.953565e+06
        2024-11-30 16:00:00  1.0800  1.0959  1.0929  1.0866  7.705551e+06
        2024-11-30 20:00:00  1.0736  1.0978  1.0862  1.0797  1.199009e+07
        2024-12-01 00:00:00  1.0373  1.0857  1.0796  1.0511  1.483185e+07
        2024-12-01 04:00:00  1.0442  1.0610  1.0515  1.0572  5.513697e+06
        2024-12-01 08:00:00  1.0556  1.0788  1.0566  1.0722  5.476993e+06
        2024-12-01 12:00:00  1.0585  1.0755  1.0721  1.0742  7.516496e+06
        2024-12-01 16:00:00  1.0565  1.0984  1.0740  1.0946  1.689854e+07
        2024-12-01 20:00:00  1.0755  1.1647  1.0947  1.1503  4.851472e+07
        2024-12-02 00:00:00  1.1264  1.1913  1.1505  1.1403  4.316623e+07
        2024-12-02 04:00:00  1.1167  1.1689  1.1402  1.1416  2.022920e+07
        2024-12-02 08:00:00  1.0568  1.1585  1.1412  1.0731  2.124469e+07
        2024-12-02 12:00:00  1.0675  1.2261  1.0725  1.2078  4.767056e+07
        2024-12-02 16:00:00  1.1558  1.2428  1.2064  1.2192  5.063293e+07
        2024-12-02 20:00:00  1.1753  1.2339  1.2186  1.2024  2.134085e+07
        2024-12-03 00:00:00  1.1841  1.3278  1.2028  1.2982  5.896306e+07
        2024-12-03 04:00:00  1.2517  1.3126  1.2983  1.2682  3.060346e+07
        2024-12-03 08:00:00  1.2310  1.3140  1.2691  1.2858  1.837665e+07
        2024-12-03 12:00:00  1.1402  1.3199  1.2858  1.1978  4.949697e+07
        2024-12-03 16:00:00  1.1686  1.2264  1.1977  1.1897  2.512853e+07
        2024-12-03 20:00:00  1.1837  1.2365  1.1895  1.1920  2.169567e+07
        2024-12-04 00:00:00  1.1809  1.2199  1.1930  1.2096  1.624852e+07
        2024-12-04 04:00:00  1.2091  1.2414  1.2095  1.2299  1.086193e+07
        2024-12-04 08:00:00  1.2173  1.2420  1.2298  1.2243  8.387761e+06
        2024-12-04 12:00:00  1.1852  1.2287  1.2242  1.1914  1.832581e+07
        2024-12-04 16:00:00  1.1407  1.1980  1.1915  1.1923  3.300019e+07
        2024-12-04 20:00:00  1.1649  1.2120  1.1923  1.1887  2.184292e+07
        2024-12-05 00:00:00  1.1163  1.2400  1.1886  1.1829  4.667123e+07
        2024-12-05 04:00:00  1.1735  1.2140  1.1830  1.2068  1.560264e+07
        2024-12-05 08:00:00  1.2055  1.2384  1.2069  1.2249  1.388446e+07
        2024-12-05 12:00:00  1.1857  1.2343  1.2248  1.2118  1.977112e+07
        2024-12-05 16:00:00  1.1861  1.2461  1.2121  1.1889  2.221459e+07
        2024-12-05 20:00:00  1.1435  1.2085  1.1891  1.1612  2.372577e+07
        2024-12-06 00:00:00  1.1530  1.1947  1.1614  1.1887  1.418680e+07
        2024-12-06 04:00:00  1.1849  1.2012  1.1887  1.1895  7.217247e+06
        2024-12-06 08:00:00  1.1502  1.1941  1.1893  1.1628  9.525318e+06
        2024-12-06 12:00:00  1.1358 

In [176]:
df_all.reset_index().groupby(['ticker']).size()

ticker
1INCH-USD    6031
AAVE-USD     6031
ADA-USD      6031
AIOZ-USD     6031
ALGO-USD     6031
AMP-USD      6031
ATOM-USD     6031
AVAX-USD     6031
BAL-USD      6031
BTC-USD      6031
COMP-USD     6031
CRO-USD      6031
CRV-USD      6031
DIA-USD      6031
DOGE-USD     6031
DOT-USD      6031
ETH-USD      6031
FET-USD      6031
FIL-USD      6031
GRT-USD      6031
ICP-USD      6031
IMX-USD      6031
KRL-USD      6020
LINK-USD     6031
LRC-USD      6031
LTC-USD      6031
MANA-USD     6031
MKR-USD      6031
OXT-USD      6031
REQ-USD      6031
RNDR-USD     6031
SHIB-USD     6031
SKL-USD      6031
SNX-USD      6031
SOL-USD      6031
STX-USD      6031
SUSHI-USD    6031
UNI-USD      6031
XLM-USD      6031
XTZ-USD      6031
dtype: int64